In this notebook, I will use torch_geometric to predict the developpement of a graph of positions through time

In [1]:
"""
I realized I am leaning towards this approach https://doi.org/10.1016/j.trc.2020.102635
"""

'\nI realized I am leaning towards this approach https://doi.org/10.1016/j.trc.2020.102635\n'

In [3]:
import torch
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import pickle

import sys
import os
from genericpath import exists

from cell_dataset import CellGraphDataset, loadDataset
from cell_model import Gatv2Predictor
from cell_utils import GraphingLoss
from cell_training import train, test_single, compute_parameters_draw

import os, psutil
process = psutil.Process(os.getpid())
print("Using : ", process.memory_info().rss // 1000000)  # in megabytes 
print("Available : ", process.memory_info().vms  // 1000000)  # in megabytes 

print(torch.cuda.is_available())

#https://github.com/clovaai/AdamP
from adamp import AdamP

sys.path.append('/home/nstillman/1_sbi_activematter/cpp_model')
try :
    import allium
except :
    print("Could not import allium")

Using :  544
Available :  43953
True
Cannot import zuko. Continuing without prior
No module named 'pycapmd'
Cannot import simulator


The data is a graph of cells having their own positions and velocity.

In the graph, we will first start by connecting all the edges, then maybe later make radius_graphs to reduce the cost of the pass through the model

In [1]:
load_all =  True #load directly from a pickle
pre_separated = False #if three subfolders already exist for train test and val

override = False #make this true to always use the same ones

extension = "_open_lt_lv"
number_of_messages = 2
size_of_messages = 64
absolute = 0
epochs = 201
distrib = "laplace"
aggr = "mean"

name_complete = extension + "_" + str(number_of_messages) + "_" + str(size_of_messages) + "_" + ("absolute" if absolute else "relative") + "_" + distrib + "_" + aggr

print(name_complete)

model_path = "models/model" + name_complete
loss_path = "models/loss" + name_complete

_open_lt_lv_2_64_relative_laplace_mean


In [4]:
data_train, data_test, data_val = loadDataset(load_all, extension, pre_separated, override)

Validation data not found


In [5]:
data_point = data_train.get(0, -1)
print(data_point[1].shape)
print(data_point[1][1,:,4])

torch.Size([99, 100, 5])
tensor([17., 10., 11.,  9., 16.,  7.,  6.,  6.,  8.,  9.,  6.,  6., 10., 10.,
         6., 13., 12., 13., 15., 11.,  9., 10., 15.,  9.,  8., 10., 12.,  6.,
        11., 13.,  7.,  7.,  8.,  9., 10., 15.,  9.,  8., 12.,  9., 12.,  9.,
        13., 11.,  7., 16., 10., 12.,  9., 10.,  7., 13., 12., 13., 12., 11.,
        10., 15., 13.,  8., 11.,  6.,  4., 12., 10.,  8., 13., 11.,  7., 12.,
         6.,  7., 10., 12., 12., 10., 10.,  6., 10., 12., 12.,  7., 14.,  6.,
         5.,  4.,  6., 11., 10., 10.,  8.,  8., 15., 12.,  5., 10., 15., 11.,
        12.,  9.])


In [7]:
new_val, edge_index, edge_attr = data_train.get_edges(data_point[1][1,:,:2].cpu(), 10, False, masks=None, previous=data_point[1][0,:,:2].cpu())

In [8]:
print(new_val.shape)
print(new_val[:,:,4])

torch.Size([1, 100, 5])
tensor([[17., 10., 11.,  9., 16.,  7.,  6.,  6.,  8.,  9.,  6.,  6., 10., 10.,
          6., 13., 12., 13., 15., 11.,  9., 10., 15.,  9.,  8., 10., 12.,  6.,
         11., 13.,  7.,  7.,  8.,  9., 10., 15.,  9.,  8., 12.,  9., 12.,  9.,
         13., 11.,  7., 16., 10., 12.,  9., 10.,  7., 13., 12., 13., 12., 11.,
         10., 15., 13.,  8., 11.,  6.,  4., 12., 10.,  8., 13., 11.,  7., 12.,
          6.,  7., 10., 12., 12., 10., 10.,  6., 10., 12., 12.,  7., 14.,  6.,
          5.,  4.,  6., 11., 10., 10.,  8.,  8., 15., 12.,  5., 10., 15., 11.,
         12.,  9.]])


Next we need to define the model that will be used :
    > input 
        (1) Graph at a particular time t (nodes having x,y,dx,dy as attributes)
        (2) Graphs up to a particular time [t-a, t] (nodes having x,y as attributes)
    > output
        (a) Graph at the immediate next time step t+1
        (b) Graph [t, t+b]
        (c) Graph at t+b
    > graph size
        (x) Fixed graph size to the most nodes possible (or above)
        (y) Unbounded graph size
            >> idea : graph walks
            >> idea : sampler

The following model will do (1ax)

In [6]:
def start(model : Gatv2Predictor, optimizer : torch.optim.Optimizer, scheduler  : torch.optim.lr_scheduler._LRScheduler,\
          data_train : CellGraphDataset, data_test : CellGraphDataset, device : torch.device, epoch : int, offset : int, grapher : GraphingLoss, save=0, save_datasets=True):
    
    loss_history_test_recursive = {"loss_mean" : [], "loss_log" : [], "loss" : []}
    loss_history_train = {"loss" : []}
    for e in range(offset, offset + epoch):

        train_loss = train(model, optimizer, scheduler, data_train, device, e, process, max_epoch=offset+epoch, distrib=distrib, aggr=aggr)
        loss_history_train["loss"] += train_loss

        #model.show_gradients()
        
        if(e == 0 and save_datasets) :
            data_train.thread = None
            data_test.thread = None
            with open("data/training" + extension + ".pkl", 'wb') as f:
                pickle.dump(data_train, f)
            with open("data/testing" + extension + ".pkl", 'wb') as f:
                pickle.dump(data_test, f)
            print("Saved datasets")
        
        test_loss_r = test_single(model, data_test, device, loss_history_test_recursive, duration=0, distrib=distrib, aggr=aggr)

        print("Epoch : ", e, "Test loss recursive : ", test_loss_r)

        grapher.plot_losses(title="Testing recursive", data=loss_history_test_recursive, length=min(50, len(data_test)), extension=name_complete + "_") 
        grapher.plot_losses(title="Training", data=loss_history_train, length=len(data_train), extension=name_complete + "_")
        
        if (e%save == 0) :    
            all_params_out, all_params_true = compute_parameters_draw(model, data_test, device, duration=-1, distrib=distrib)
            grapher.plot_params(all_params_out, all_params_true, e, extension=name_complete)
        
        if (save and (e%save == 0 or e == epoch-1)) :
            torch.save(model.state_dict(), model_path + str(e) + ".pt")

In [7]:
load = False

epoch_to_load = 0

model = Gatv2Predictor(in_channels=12, out_channels=8, hidden_channels=size_of_messages, dropout=0.05, edge_dim=2, messages=number_of_messages, wrap=data_train.wrap, absolute=absolute)
#model = Gatv2Predictor(in_channels=9, out_channels=4, hidden_channels=32, dropout=0.01, edge_dim=2, messages=5, wrap=True)
#model = Gatv2PredictorDiscr(in_channels=9, out_channels=4, hidden_channels=16, dropout=0.01, edge_dim=2, messages=5, wrap=True)
#model = ConvPredictor(in_channels=16, out_channels=8, hidden_channels=size_of_messages, dropout=0.05, edge_dim=2, messages=number_of_messages, wrap=data_train.wrap, absolute=absolute, heads=4)

if exists(model_path + str(epoch_to_load) + ".pt") and load :
    model.load_state_dict(torch.load(model_path + str(epoch_to_load) + ".pt"))
    print("Loaded model")

In [8]:
#might want to investigate AdamP 
optimizer = AdamP(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-3, weight_decay=5e-3, delta=0.1, wd_ratio=0.1, nesterov=True)
scheduler = CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=10, T_mult=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
grapher = GraphingLoss()

model = model.to(device)

In [10]:
start(model, optimizer, scheduler, data_train, data_test, device, \
        epochs, epoch_to_load, grapher=grapher, save=50, save_datasets=False)

In [3]:
from torch_geometric.nn import knn_graph
import torch

square = torch.tensor([[0,0], [0,1], [1,2], [1,1]], dtype=torch.float)

square_repeter = square.unsqueeze(0).repeat(5,1,1)

edge_index = knn_graph(square_repeter.view(-1,2), k = 3, loop=True, flow="source_to_target")

T = square_repeter.shape[0]
N = square_repeter.shape[1]

degree = torch.zeros((T,N,1))
for i in range(T) :
    for j in range(N) :
        degree[i, j, 0] = torch.sum(edge_index[1, :] == i*N+j)

print(degree)

print(edge_index)

tensor([[[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]]])
tensor([[ 0, 16, 12, 17,  1, 13, 14,  2, 10, 11, 15, 19,  0, 16, 12, 17,  1, 13,
         14,  2, 10, 11, 15, 19,  0, 16, 12, 17,  1, 13, 14,  2, 10, 11, 15, 19,
          0, 16, 12, 17,  1, 13, 14,  2, 10, 11, 15, 19,  0, 16, 12, 17,  1, 13,
         14,  2, 10, 11, 15, 19],
        [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,
          6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11, 11,
         12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17,
         18, 18, 18, 19, 19, 19]])


In [5]:
import torch

a = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12]).view(1,3,4).repeat(15,1,1).to(torch.float)


print(a[-6:].shape)

print(a.mean(dim=(0,1)).shape)

torch.Size([6, 3, 4])
torch.Size([4])
